### 1. query keyword 추출

In [1]:
QUERY = "너에 대해 네 가지 특징을 말해 보세요"

In [2]:

from mecab import MeCab as mecab
mecab = mecab()

##### `mecab-co-dic 품사 태그`

    - NNG : 일반명사
    - NNP : 고유명사
    - NNB : 의존명사
    - NNBC : 단위를 나타내는 명사
    - NR : 수사
    - NP : 대명사

In [3]:
def get_pos_tagging(query):
    print(f" 원 쿼리 -> {query}")
    mecab_result = mecab.pos(query)
    print(f" mecab pos tagging -> {mecab_result}")
    result = []
    combined_word = ''

    cur = 0
    prev_pos = None
    while cur < len(mecab_result):
        word, pos = mecab_result[cur]
        if pos == 'SN':
            combined_word += word
            nxt = cur + 1
            while nxt < len(mecab_result) and mecab_result[nxt][1] == 'SN':
                combined_word += mecab_result[nxt][0]
                nxt += 1
                
            while nxt < len(mecab_result) and mecab_result[nxt][1] in ['NNG', 'NNP', 'NNB', 'NNBC', 'NR', 'NP']:
                if mecab_result[nxt-1][1] in ['NNG', 'NNP', 'NNB', 'NNBC', 'NR', 'NP']:
                    break
                combined_word += mecab_result[nxt][0]
                nxt += 1
            result.append(combined_word)
            combined_word = ''
            cur = nxt
        elif pos in ['MAG', 'NNG', 'NNP', 'NNB', 'NNBC', 'NR', 'NP']:
            combined_word = word
            result.append(combined_word)
            nxt = cur + 1
            combined_word = ''
            cur = nxt
        else:
            cur += 1

    print(f" 키워드 추출 -> {result}")
    return result

In [4]:
result = get_pos_tagging(QUERY)

 원 쿼리 -> 너에 대해 네 가지 특징을 말해 보세요
 mecab pos tagging -> [('너', 'NP'), ('에', 'JKB'), ('대해', 'VV+EC'), ('네', 'MM'), ('가지', 'NNBC'), ('특징', 'NNG'), ('을', 'JKO'), ('말', 'NNG'), ('해', 'XSV+EC'), ('보', 'VX'), ('세요', 'EP+EF')]
 키워드 추출 -> ['너', '가지', '특징', '말']


### 2. 검색어를 가져와서 조합 (random)

In [5]:
import random

In [6]:
keyword_combination_1= '+'.join(random.sample(result, len(result)))
print(keyword_combination_1)

너+가지+말+특징


In [7]:
keyword_combination_2= '+'.join(random.sample(result, len(result)))
print(keyword_combination_2)

말+가지+특징+너


In [8]:
keyword_combination_3= '+'.join(random.sample(result, len(result)))
print(keyword_combination_3)

특징+너+가지+말


### 3. 검색어 조합을 통해서 rss 활용해 뉴스 기사 load

In [9]:
import feedparser
import ssl
import pandas as pd
ssl._create_default_https_context = ssl._create_unverified_context


In [10]:
def search_rss_keyword(keyword_combination):
    corps = set()
    rss_url = f"https://news.google.com/rss/search?q={keyword_combination}&hl=ko&gl=KR&ceid=KR:ko"
    # rss_url = f"https://news.google.com/rss/search?q={keyword}"
    feed = feedparser.parse(rss_url)

    results = []
    for entry in feed.entries:
        result = {
            'title': entry.title,
            'link': entry.link,
            'published': entry.published,
            'corp' : entry.title.split(' - ')[1]
        }
        results.append(result)
        corps.add(entry.title.split(' - ')[1])
        
    return results
    

In [11]:
case1 = search_rss_keyword(keyword_combination_1)
case2 = search_rss_keyword(keyword_combination_2)
case3 = search_rss_keyword(keyword_combination_3)

In [12]:
url_call_df_1= pd.DataFrame(case1)
url_call_df_1

,title,link,published,corp
0,“너 T야?” MBTI에서 T가 문제적 문자된 까닭은 - 한겨레21,https://news.google.com/rss/articles/CBMiNmh0d...,"Wed, 30 Aug 2023 07:00:00 GMT",한겨레21
1,사랑받지 못하는 사람들의 특징 1가지 - 인천in 시민의 손으로 만드는 인터넷신문,https://news.google.com/rss/articles/CBMiO2h0d...,"Mon, 18 Dec 2023 08:00:00 GMT",인천in 시민의 손으로 만드는 인터넷신문
2,진도 아리랑 계열 20여 종 - 오마이뉴스,https://news.google.com/rss/articles/CBMiRGh0d...,"Sat, 10 Sep 2022 07:00:00 GMT",오마이뉴스
3,"“너 몇 살이야” 한국에서 ‘만 나이’ 통일 어려운 이유, 국어학자 분석해보니 - YTN",https://news.google.com/rss/articles/CBMiMWh0d...,"Mon, 04 Jul 2022 07:00:00 GMT",YTN
4,”넌 너무 예민해” 가스라이팅 하는 사람의 특징과 6가지 가장 자주 하는 말 (전문...,https://news.google.com/rss/articles/CBMiQGh0d...,"Wed, 28 Apr 2021 07:00:00 GMT",허프포스트코리아
5,"너는 참, 같은 말을 해도 > 뉴스 - 비즈투데이",https://news.google.com/rss/articles/CBMiOmh0d...,"Wed, 27 Apr 2022 07:00:00 GMT",비즈투데이
6,고양이를 상처 입히는 집사의 말과 행동 5가지 - 비마이펫 라이프,https://news.google.com/rss/articles/CBMiHmh0d...,"Thu, 13 Feb 2020 08:00:00 GMT",비마이펫 라이프
7,이혼하는 부부들의 대화패턴 4가지 - 마음건강 길,https://news.google.com/rss/articles/CBMiOWh0d...,"Mon, 17 Aug 2020 07:00:00 GMT",마음건강 길
8,아무말 없이 가만히 있어도 1도 어색하지 않은 '진짜 친구'들만이 가지는 특징 5가...,https://news.google.com/rss/articles/CBMiJWh0d...,"Thu, 30 Jul 2020 07:00:00 GMT",인사이트
9,천재+4차원 특징 모두 겸비한 매력 만점 AB형 남자 특징 5가지 - 인사이트,https://news.google.com/rss/articles/CBMiJWh0d...,"Tue, 06 Aug 2019 07:00:00 GMT",인사이트


In [13]:
url_call_df_2= pd.DataFrame(case2)
url_call_df_2

,title,link,published,corp
0,“너 T야?” MBTI에서 T가 문제적 문자된 까닭은 - 한겨레21,https://news.google.com/rss/articles/CBMiNmh0d...,"Wed, 30 Aug 2023 07:00:00 GMT",한겨레21
1,“일할 의욕꺾는 회사는 이랬다” 9가지 유형 | 컴퍼니 타임스의 비즈니스 뉴스 | ...,https://news.google.com/rss/articles/CBMiLmh0d...,"Mon, 29 Aug 2022 07:00:00 GMT",컴퍼니 타임스
2,”넌 너무 예민해” 가스라이팅 하는 사람의 특징과 6가지 가장 자주 하는 말 (전문...,https://news.google.com/rss/articles/CBMiQGh0d...,"Wed, 28 Apr 2021 07:00:00 GMT",허프포스트코리아
3,"“너 몇 살이야” 한국에서 ‘만 나이’ 통일 어려운 이유, 국어학자 분석해보니 - YTN",https://news.google.com/rss/articles/CBMiMWh0d...,"Mon, 04 Jul 2022 07:00:00 GMT",YTN
4,4가지 상징으로 풀어내는 대화의 심리학 - 연합뉴스,https://news.google.com/rss/articles/CBMiL2h0d...,"Wed, 20 Jan 2021 08:00:00 GMT",연합뉴스
5,고양이를 상처 입히는 집사의 말과 행동 5가지 - 비마이펫 라이프,https://news.google.com/rss/articles/CBMiHmh0d...,"Thu, 13 Feb 2020 08:00:00 GMT",비마이펫 라이프
6,우리는 왜 커뮤니케이션하는가? - ZD넷 코리아,https://news.google.com/rss/articles/CBMiK2h0d...,"Tue, 03 Nov 2020 08:00:00 GMT",ZD넷 코리아
7,이혼하는 부부들의 대화패턴 4가지 - 마음건강 길,https://news.google.com/rss/articles/CBMiOWh0d...,"Mon, 17 Aug 2020 07:00:00 GMT",마음건강 길
8,남자를 피곤하게 하는 여자친구유형 4가지 - 한겨레,https://news.google.com/rss/articles/CBMiP2h0d...,"Sat, 19 Oct 2019 07:00:00 GMT",한겨레
9,아무말 없이 가만히 있어도 1도 어색하지 않은 '진짜 친구'들만이 가지는 특징 5가...,https://news.google.com/rss/articles/CBMiJWh0d...,"Thu, 30 Jul 2020 07:00:00 GMT",인사이트


In [14]:
url_call_df_3= pd.DataFrame(case3)
url_call_df_3

,title,link,published,corp
0,“너 T야?” MBTI에서 T가 문제적 문자된 까닭은 - 한겨레21,https://news.google.com/rss/articles/CBMiNmh0d...,"Wed, 30 Aug 2023 07:00:00 GMT",한겨레21
1,“일할 의욕꺾는 회사는 이랬다” 9가지 유형 | 컴퍼니 타임스의 비즈니스 뉴스 | ...,https://news.google.com/rss/articles/CBMiLmh0d...,"Mon, 29 Aug 2022 07:00:00 GMT",컴퍼니 타임스
2,”넌 너무 예민해” 가스라이팅 하는 사람의 특징과 6가지 가장 자주 하는 말 (전문...,https://news.google.com/rss/articles/CBMiQGh0d...,"Wed, 28 Apr 2021 07:00:00 GMT",허프포스트코리아
3,'멘탈 갑'인 사람들이 자주 하는 말 7가지 - 머니투데이,https://news.google.com/rss/articles/CBMiN2h0d...,"Sat, 04 May 2019 07:00:00 GMT",머니투데이
4,"“너 몇 살이야” 한국에서 ‘만 나이’ 통일 어려운 이유, 국어학자 분석해보니 - YTN",https://news.google.com/rss/articles/CBMiMWh0d...,"Mon, 04 Jul 2022 07:00:00 GMT",YTN
5,고양이를 상처 입히는 집사의 말과 행동 5가지 - 비마이펫 라이프,https://news.google.com/rss/articles/CBMiHmh0d...,"Thu, 13 Feb 2020 08:00:00 GMT",비마이펫 라이프
6,이혼하는 부부들의 대화패턴 4가지 - 마음건강 길,https://news.google.com/rss/articles/CBMiOWh0d...,"Mon, 17 Aug 2020 07:00:00 GMT",마음건강 길
7,아무말 없이 가만히 있어도 1도 어색하지 않은 '진짜 친구'들만이 가지는 특징 5가...,https://news.google.com/rss/articles/CBMiJWh0d...,"Thu, 30 Jul 2020 07:00:00 GMT",인사이트
8,친구랑 있으면 ‘인싸’인데 밖에서는 유난히 ‘낯가림’ 심한 친구들의 5가지 특징 -...,https://news.google.com/rss/articles/CBMiJWh0d...,"Sat, 13 Feb 2021 08:00:00 GMT",인사이트
9,성격 까칠한 친구 때문에 확 떨어진 '자존감' 끌어올리는 7가지 방법 - 인사이트,https://news.google.com/rss/articles/CBMiJWh0d...,"Wed, 18 Jul 2018 07:00:00 GMT",인사이트


In [15]:
final_url_df = pd.merge(pd.merge(url_call_df_1, url_call_df_2), url_call_df_3)
final_url_df

,title,link,published,corp
0,“너 T야?” MBTI에서 T가 문제적 문자된 까닭은 - 한겨레21,https://news.google.com/rss/articles/CBMiNmh0d...,"Wed, 30 Aug 2023 07:00:00 GMT",한겨레21
1,"“너 몇 살이야” 한국에서 ‘만 나이’ 통일 어려운 이유, 국어학자 분석해보니 - YTN",https://news.google.com/rss/articles/CBMiMWh0d...,"Mon, 04 Jul 2022 07:00:00 GMT",YTN
2,”넌 너무 예민해” 가스라이팅 하는 사람의 특징과 6가지 가장 자주 하는 말 (전문...,https://news.google.com/rss/articles/CBMiQGh0d...,"Wed, 28 Apr 2021 07:00:00 GMT",허프포스트코리아
3,고양이를 상처 입히는 집사의 말과 행동 5가지 - 비마이펫 라이프,https://news.google.com/rss/articles/CBMiHmh0d...,"Thu, 13 Feb 2020 08:00:00 GMT",비마이펫 라이프
4,이혼하는 부부들의 대화패턴 4가지 - 마음건강 길,https://news.google.com/rss/articles/CBMiOWh0d...,"Mon, 17 Aug 2020 07:00:00 GMT",마음건강 길
5,아무말 없이 가만히 있어도 1도 어색하지 않은 '진짜 친구'들만이 가지는 특징 5가...,https://news.google.com/rss/articles/CBMiJWh0d...,"Thu, 30 Jul 2020 07:00:00 GMT",인사이트


### 4. ML 기반 유사도 높은 문서 5개 가져오기

In [16]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
from sentence_transformers import SentenceTransformer

/Users/geonheekim/Desktop/rss/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
model = SentenceTransformer('sentence-transformers/stsb-xlm-r-multilingual')

/Users/geonheekim/Desktop/rss/venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/geonheekim/Desktop/rss/venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [18]:
train_data = final_url_df.copy()

In [19]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.title), axis = 1)

In [20]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [21]:
train_data

,title,link,published,corp,embedding
0,“너 T야?” MBTI에서 T가 문제적 문자된 까닭은 - 한겨레21,https://news.google.com/rss/articles/CBMiNmh0d...,"Wed, 30 Aug 2023 07:00:00 GMT",한겨레21,"[0.20299917, 0.02975824, 1.0284082, -0.2123534..."
1,"“너 몇 살이야” 한국에서 ‘만 나이’ 통일 어려운 이유, 국어학자 분석해보니 - YTN",https://news.google.com/rss/articles/CBMiMWh0d...,"Mon, 04 Jul 2022 07:00:00 GMT",YTN,"[-0.4824589, -0.12705529, 1.0666262, -0.408913..."
2,”넌 너무 예민해” 가스라이팅 하는 사람의 특징과 6가지 가장 자주 하는 말 (전문...,https://news.google.com/rss/articles/CBMiQGh0d...,"Wed, 28 Apr 2021 07:00:00 GMT",허프포스트코리아,"[-0.11226704, 0.022798719, 0.54279375, 0.39832..."
3,고양이를 상처 입히는 집사의 말과 행동 5가지 - 비마이펫 라이프,https://news.google.com/rss/articles/CBMiHmh0d...,"Thu, 13 Feb 2020 08:00:00 GMT",비마이펫 라이프,"[0.26274773, 0.28402302, 0.60084033, -0.255276..."
4,이혼하는 부부들의 대화패턴 4가지 - 마음건강 길,https://news.google.com/rss/articles/CBMiOWh0d...,"Mon, 17 Aug 2020 07:00:00 GMT",마음건강 길,"[-0.09100527, 0.8048264, 0.96476316, 0.1171435..."
5,아무말 없이 가만히 있어도 1도 어색하지 않은 '진짜 친구'들만이 가지는 특징 5가...,https://news.google.com/rss/articles/CBMiJWh0d...,"Thu, 30 Jul 2020 07:00:00 GMT",인사이트,"[-0.23132731, 0.18013029, 0.19133441, 0.238454..."


In [22]:
query = QUERY

embedding = model.encode(query)
train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
sim_result_df = train_data.sort_values(by='score', ascending=False)[:5].reset_index(drop=True)
sim_result_df.loc[:5]

,title,link,published,corp,embedding,score
0,이혼하는 부부들의 대화패턴 4가지 - 마음건강 길,https://news.google.com/rss/articles/CBMiOWh0d...,"Mon, 17 Aug 2020 07:00:00 GMT",마음건강 길,"[-0.09100527, 0.8048264, 0.96476316, 0.1171435...",0.460520
1,“너 T야?” MBTI에서 T가 문제적 문자된 까닭은 - 한겨레21,https://news.google.com/rss/articles/CBMiNmh0d...,"Wed, 30 Aug 2023 07:00:00 GMT",한겨레21,"[0.20299917, 0.02975824, 1.0284082, -0.2123534...",0.372283
2,고양이를 상처 입히는 집사의 말과 행동 5가지 - 비마이펫 라이프,https://news.google.com/rss/articles/CBMiHmh0d...,"Thu, 13 Feb 2020 08:00:00 GMT",비마이펫 라이프,"[0.26274773, 0.28402302, 0.60084033, -0.255276...",0.371692
3,”넌 너무 예민해” 가스라이팅 하는 사람의 특징과 6가지 가장 자주 하는 말 (전문...,https://news.google.com/rss/articles/CBMiQGh0d...,"Wed, 28 Apr 2021 07:00:00 GMT",허프포스트코리아,"[-0.11226704, 0.022798719, 0.54279375, 0.39832...",0.281781
4,아무말 없이 가만히 있어도 1도 어색하지 않은 '진짜 친구'들만이 가지는 특징 5가...,https://news.google.com/rss/articles/CBMiJWh0d...,"Thu, 30 Jul 2020 07:00:00 GMT",인사이트,"[-0.23132731, 0.18013029, 0.19133441, 0.238454...",0.260740


### 5. 뉴스 기사 크롤링

In [23]:
import requests
from bs4 import BeautifulSoup, Comment
import feedparser
from newspaper import Article
import datetime
from fake_useragent import UserAgent

In [24]:
class NewsCrawler():
    def __init__(self):
        self.user_agent = UserAgent()
        self.headers = None

    def extract_tag(self, item, tag):
        [elem.extract() for elem in item(tag)]


    def extract_comments(self, item):
        for element in item(text=lambda text: isinstance(text, Comment)):
            element.extract()
            
    def get_headers(self, shuffle_header=False):
        if self.headers is None or shuffle_header:
            self.headers = {
                'User-Agent': self.user_agent.random,
            }
        return self.headers


    def crawl_url(self, url, shuffle_header=False, language='ko'):
        try:
            # article title
            article = Article(url, language=language)
            article.download()
            article.parse()
            if not article.url:
                print("[ERROR] no url found..")
                return None        

            data = {}
            data['title'] = article.title
            data['image'] = article.meta_img
            data['crawl_url'] = url
            data['url'] = article.url

            print(article.url)
            if article.publish_date:
                data['published_at'] = article.publish_date
            else:
                data['published_at'] = datetime.datetime.now()

            res = requests.get(url, headers=self.get_headers(shuffle_header=shuffle_header))
            soup = BeautifulSoup(res.text, 'html.parser')

            for script in soup(['script', 'style']):
                script.decompose()

            body = soup.find("div", itemprop="articleBody")

            if body is None:
                body = soup.select('#articleBodyContents')
                if len(body) > 0:
                    body = body[0]
                    self.extract_comments(body)
                    self.extract_tag(body, 'img')
                    self.extract_tag(body, 'div')
                    body = str(body)
                else:
                    body = article.text
                    body = body.replace('\n','<br>')
            else:
                self.extract_comments(body)
                self.extract_tag(body, 'img')
                self.extract_tag(body, 'div')
                body = str(body)

            data['text'] = article.text
            data['body'] = body
            data['language'] = language

        except Exception as e:
            print(e)
            data = None

        return data
    
    def crawl_rss(self, rss_url, shuffle_header=False, language='ko'):
        feed = feedparser.parse(rss_url)
        data_list = []
        for post in feed.entries:
            if post.link:
                data_list.append(self.crawl_url(post.link, shuffle_header, language))

        return data_list

In [25]:
content_df = pd.DataFrame()

In [26]:
c = NewsCrawler()

In [27]:
for idx, row in sim_result_df[:10].iterrows():
    tmp = c.crawl_url(row['link'])
    length = len(content_df)+1
    
    try :
        content_df.loc[length, 'title'] = tmp['title']
        content_df.loc[length, 'url'] = tmp['url']
        content_df.loc[length, 'published_at'] = tmp['published_at']
        content_df.loc[length, 'text'] = tmp['text'].replace('\n', '')
    except:
        pass
    
content_df

https://news.google.com/rss/articles/CBMiOWh0dHBzOi8vd3d3Lm1pbmRnaWwuY29tL25ld3MvYXJ0aWNsZVZpZXcuaHRtbD9pZHhubz02OTY2OdIBPGh0dHBzOi8vd3d3Lm1pbmRnaWwuY29tL25ld3MvYXJ0aWNsZVZpZXdBbXAuaHRtbD9pZHhubz02OTY2OQ?oc=5
https://news.google.com/rss/articles/CBMiNmh0dHBzOi8vaDIxLmhhbmkuY28ua3IvYXJ0aS9jdWx0dXJlL2N1bHR1cmUvNTQyMzYuaHRtbNIBAA?oc=5


/var/folders/14/q73v_xlx3_x8kr742lgx5_sh0000gn/T/ipykernel_35698/1835947185.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-05-10 15:16:43.279598' has dtype incompatible with datetime64[us, tzoffset(None, 32400)], please explicitly cast to a compatible dtype first.
  content_df.loc[length, 'published_at'] = tmp['published_at']


https://news.google.com/rss/articles/CBMiHmh0dHBzOi8vbXlwZXRsaWZlLmNvLmtyLzc3NTM1L9IBAA?oc=5
https://news.google.com/rss/articles/CBMiQGh0dHBzOi8vd3d3Lmh1ZmZpbmd0b25wb3N0LmtyL25ld3MvYXJ0aWNsZVZpZXcuaHRtbD9pZHhubz0xMDk0MDHSAQA?oc=5
https://news.google.com/rss/articles/CBMiJWh0dHBzOi8vd3d3Lmluc2lnaHQuY28ua3IvbmV3cy8yOTQ3MTPSAQA?oc=5


,title,url,published_at,text
1,이혼하는 부부들의 대화패턴 4가지,https://news.google.com/rss/articles/CBMiOWh0d...,2020-08-17 12:07:07+09:00,세계적인 부부 문제 전문가 존 가트맨 미 워싱턴대 심리학과 석좌교수는 지난 40년간...
2,“너 T야?” MBTI에서 T가 문제적 문자된 까닭은,https://news.google.com/rss/articles/CBMiNmh0d...,2024-05-10 15:16:43.279598,"“너 티(T)야?” 요즘 번화가 횡단보도를 건너다가, 맥줏집 옆 테이블에서, 귀갓길..."
3,고양이 상처 입히는 집사의 말과 행동 5가지,https://news.google.com/rss/articles/CBMiHmh0d...,2020-02-13 01:34:03+00:00,고양이 상처 입히는 집사의 말과 행동 5가지고양이 상처 입히는 집사의 말과 행동 5...
4,”넌 너무 예민해” 가스라이팅 하는 사람이 가장 자주 하는 6가지 말과 대처법 (심...,https://news.google.com/rss/articles/CBMiQGh0d...,2021-04-28 12:07:31+09:00,ⓒShotPrime via Getty Images가스라이팅은 상대의 감정을 휘두르며...
5,아무말 없이 가만히 있어도 1도 어색하지 않은 '진짜 친구'들만이 가지는 특징 5가지,https://news.google.com/rss/articles/CBMiJWh0d...,2020-07-30 17:58:16+09:00,기사와 관련 없는 자료 사진 / Naver TV '플레이리스트'[인사이트] 디지털뉴...


### 6. 크롤링한 기사 전처리 (작업중)

In [28]:
import re

In [29]:
# custom_words = ['기자', '일보', '앵커']   

# for idx, row in url_df.loc[:1].iterrows():
#     url = row['url']
#     print(row['text'])
#     text = row['text'].replace('\n', '')
#     pattern1 = r'\b(?:' + '|'.join(list(corps)) + r')\b'
#     preprocessed_text = re.sub(pattern1, '', text)
#     pattern2 = r'[^\w\s,~.\'"]'
#     preprocessed_text = re.sub(pattern2, '', preprocessed_text)
#     pattern3 = r'\b(?:\w+ 앵커|\w+ 기자|\w+일보)\b'
#     preprocessed_text = re.sub(pattern3, '', preprocessed_text)

#     print(preprocessed_text)

### 7. LLM 요약

In [30]:
from dotenv import load_dotenv

load_dotenv()

True

In [31]:
import json

### 7-1. LLM (Retrieval)

In [32]:
from langchain_community.document_loaders import JSONLoader
from pprint import pprint

In [33]:
result_data_json= content_df.to_json(orient='records', force_ascii=False, date_format='iso')

In [34]:
result_json = json.loads(result_data_json)
pprint(result_json)

[{'published_at': '2020-08-17T03:07:07.000Z',
  'text': '세계적인 부부 문제 전문가 존 가트맨 미 워싱턴대 심리학과 석좌교수는 지난 40년간 부부 3천 쌍의 상호작용을 비디오로 '
          '찍어 상세히 분석한 결과 이혼으로 가는 부부에게 공통적인 대화패턴 4가지가 있음을 발견했다. 바로 비난, 방어, 경멸, '
          '담쌓기였다.① 비난“당신은 말이야~", “너는 왜 항상~", “당신은 도대체~" 등의 표현 등이다.이런 어투는 ‘나’는 '
          '문제없고 ‘너’가 문제가 있는 사람으로 인격적으로 비난하는 식으로 상대방에게 받아들여져 감정이 상한 대화가 이어지기 '
          '마련이다.② 방어(or 역공)“그런 너는 뭐 잘했냐~", “한번 따져볼까~"식으로 서로의 잘못을 따지기 시작하다 문제의 '
          '본질을 벗어나 과거의 온갖 문제까지 끄집어내며 상황을 증폭시키고 끝없는 ‘누가 옳으냐’ 공방으로 이어지게 된다.③ '
          '경멸“그래 너 잘났다", “너나 잘하세요", “주제 파악도 못하고", “복에 겨웠구나", “재수없게" 등의 표현으로 '
          '이어진다. 심하면 상소리, 욕설, 폭언까지 가세한다. 이 단계에서는 서로 인격적으로 모욕을 주는 상황이며, 가트맨에 '
          '의하면 이런 경멸은 상대방의 신체 면역력을 급격히 파괴시켜 감염성 질병에 걸릴 확률을 높인다고 한다.④ 담 쌓기상대방이 '
          '하는 말이 마음에 들지 않을 때, 아무런 반응도 보이지 않는 것을 말한다. 이는 그저 말에 대응을 하지 않는 것뿐 아니라 '
          '눈 마주치지 않기, 전화 안받기, 각방 쓰기, 각자 먹기 등의 모습을 보이며, 점차 심해지면 외박, 가출, 그리고 별거로 '
          '이어지게 된다.가트맨 교수는 부부간에 이 4단계의 대화형태가 순환되기 시작하면 마침내 이혼을 하게 된다고 했다.MIT에서 '


#### 7-2. 그냥 요약

In [35]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

In [36]:
model = ChatOpenAI(model='gpt-3.5-turbo',
                   temperature=0)

In [37]:
class SummaryObject(BaseModel):
    title: str = Field(description="본문의 title")
    url: str = Field(description="본문에 해당하는 url")
    summary: str = Field(description="본문을 요약한 텍스트")

In [38]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import ConversationChain

In [39]:
parser = JsonOutputParser(pydantic_object=SummaryObject)

In [43]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Your role is to professionally summarize news articles."
            "Summarizes news text content related to a given query."
            "We take into account several constraints during the summarizing process."
            "If you ask me to describe you, please explain in Korean that you are a chatbot created by Goyo and whose role is to summarize various news information searched on Google."            ),
            (
            "user",
            "Please summarize the sentences according to the following requests."
            "\nRequest:\n"
            "Please summarize the data in 1 to 3 lines as requested below."
            "\nRequest:\n"
            "You are a chatbot specializing in news summaries. When asked for information about yourself, just introduce yourself."
            "For weather-related inquiries, please summarize the published_at closest to {today}."
            "Please indicate the URL of the referenced content as the source."
            "Only content that can answer the provided query should be summarized in Korean."
            "Be sure to include the URL related to the referenced content as the source."
            "\n\nQuery: {query}\n\nContext: {context}\n\nSummary:",
                    ),
    ]
)
chain = prompt | llm
response= chain.invoke({"query": QUERY, "context" : result_json, "today" : datetime.date.today()})

print(response.content)

저는 Goyo가 만든 챗봇으로 구글에서 검색한 다양한 뉴스 정보를 요약하는 역할을 합니다.
